# Assignment 3 - Multi-class Classification and Neural Network
> **FULL MARKS = 10**

In this assignment, you are going to implement your own neural network to do multi-class classification. We use one-vs-all strategy here by training multiple binary classifiers (one for each class).

Please notice that you can only use numpy and scipy.optimize.minimize. **No** library versions of other method are allowed.  . Follow the instructions, you will need to fill the blanks to make it functional. The process is similar to the previous assignment. 

In [186]:
from sklearn import datasets
from scipy.optimize import minimize
import numpy as np

In [187]:
def load_dataset():
    iris = datasets.load_iris()
    X = iris.data
    y = iris.target
    
    return X, y

In [188]:
def train_test_split(X, y):
    idx = np.arange(len(X))
    train_size = int(len(X) * 2/3)
    np.random.shuffle(idx)
    X = X[idx]
    y = y[idx]
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]
    
    return X_train, X_test, y_train, y_test

In [189]:
def init_weights(num_in, num_out):
    '''
    :param num_in: the number of input units in the weight array
    :param num_out: the number of output units in the weight array
    '''

    # Note that 'W' contains both weights and bias, you can consider W[0, :] as bias
    W = np.zeros((1 + num_in, num_out))
    
    ###################################################################################
    # Full Mark: 1                                                                    #
    # TODO:                                                                           #
    # Implement Xavier/Glorot uniform initialization                                  #
    #                                                                                 #
    # Hint: you can find the reference here:                                          #
    # https://www.tensorflow.org/api_docs/python/tf/keras/initializers/GlorotUniform  #
    ###################################################################################

    lim = np.sqrt(6 / (num_in + num_out))
    for i in range(0, num_in):
      for j in range(num_out):
        W[i, j] = np.random.uniform(-lim, lim)

    ###################################################################################
    #                       END OF YOUR CODE                                          #
    ###################################################################################
    
    return W

In [190]:
def sigmoid(x):
    '''
    :param x: input
    '''

    ###################################################################################
    # Full Mark: 0.5                                                                  #
    # TODO:                                                                           #
    # Implement sigmoid function:                                                     #
    #                             sigmoid(x) = 1/(1+e^(-x))                           #
    ###################################################################################

    res = 1 / (1 + np.exp(-x))

    ###################################################################################
    #                       END OF YOUR CODE                                          #
    ###################################################################################

    return res

In [191]:
def tanh(x):
    '''
    :param x: input
    '''

    ###################################################################################
    # Full Mark: 0.5                                                                  #
    # TODO:                                                                           #
    # Implement tanh function:                                                        #
    #                     tanh(x) = (e^x-e^(-x)) / (e^x+e^(-x))                       #
    ###################################################################################

    res = (np.exp(x) - np.exp(-x)) / (np.exp(x) + np.exp(-x))

    ###################################################################################
    #                       END OF YOUR CODE                                          #
    ###################################################################################

    return res

In [192]:
def sigmoid_gradient(x):
    '''
    :param x: input
    '''

    ###################################################################################
    # Full Mark: 1                                                                    #
    # TODO:                                                                           #
    # Computes the gradient of the sigmoid function evaluated at x.                   #
    #                                                                                 #
    ###################################################################################

    p = sigmoid(x)
    grad = p * (1 - p)

    ###################################################################################
    #                       END OF YOUR CODE                                          #
    ###################################################################################

    return grad

In [193]:
def tanh_gradient(x):
    '''
    :param x: input
    '''

    ###################################################################################
    # Full Mark: 1                                                                    #
    # TODO:                                                                           #
    # Computes the gradient of the tanh function evaluated at x.                      #
    #                                                                                 #
    ###################################################################################

    grad = 1.0 - (tanh(x) ** 2)

    ###################################################################################
    #                       END OF YOUR CODE                                          #
    ###################################################################################

    return grad

In [194]:
def forward(W, X):
    '''
    :param W: weights (including biases) of the neural network. It is a list containing both W_hidden and W_output.
    :param X: input. Already added one additional column of all "1"s.
    '''

    # Shape of W_hidden: [num_feature+1, num_hidden]
    # Shape pf W_output: [num_hidden+1, num_output]
    W_hidden, W_output = W

    ###################################################################################
    # Full Mark: 1                                                                    #
    # TODO:                                                                           #
    # Implement the forward pass. You need to compute four values:                    #
    # z_hidden, a_hidden, z_output, a_output                                          #
    #                                                                                 #
    # Note that our neural network consists of three layers:                          #
    # Input -> Hidden -> Output                                                       #
    # The activation function in hidden layer is 'tanh'                               #
    # The activation function in output layer is 'sigmoid'                            #
    ###################################################################################

    #print(X[1])
    #print("X Shape:", X.shape)
    #print("W Hidden Shape:", W_hidden.shape)

    z_hidden = np.dot(X, W_hidden)
    a_hidden = tanh(z_hidden)

    #print("Z Hidden Shape:", z_hidden.shape)
    #print("A Hidden Shape:", a_hidden.shape)
    #print("W Output Shape:", W_output.shape)

    a_hidden =  np.concatenate([np.ones((X.shape[0], 1)), a_hidden], axis=1) # add one row to a_hidden cuz Shape of W_output: [num_hidden+1, num_output]
    z_output = np.dot(a_hidden, W_output)
    a_output = sigmoid(z_output)

    #print("Z Output Shape:", z_output.shape)
    #print("A Output Shape:", a_output.shape)
    
    ###################################################################################
    #                       END OF YOUR CODE                                          #
    ###################################################################################

    # z_hidden is the raw output of hidden layer, a_hidden is the result after performing activation on z_hidden
    # z_output is the raw output of output layer, a_output is the result after performing activation on z_output
    return {'z_hidden': z_hidden, 'a_hidden': a_hidden,
            'z_output': z_output, 'a_output': a_output}

In [195]:
def loss_function(W, X, y, num_feature, num_hidden, num_output, L2_lambda):
    '''
    :param W: a 1D array containing all weights and biases.
    :param X: input
    :param y: labels of X
    :param num_feature: number of features in X
    :param num_hidden: number of hidden units
    :param num_output: number of output units
    :param L2_lambda: the coefficient of regularization term
    '''

    m = y.size

    # Reshape W back into the parameters W_hidden and W_output
    #print(W.shape)
    W_hidden = np.reshape(W[:num_hidden * (num_feature + 1)],
                          ((num_feature + 1), num_hidden))

    W_output = np.reshape(W[(num_hidden * (num_feature + 1)):],
                          ((num_hidden + 1), num_output))

    # Initialize grads
    W_hidden_grad = np.zeros(W_hidden.shape)
    W_output_grad = np.zeros(W_output.shape)
    #test_grads = np.concatenate([W_hidden_grad.ravel(), W_output_grad.ravel()])
    #print("W HIDDEN GRAD:", W_hidden_grad.shape)
    #print("W OUTPUT GRAD:", W_output_grad.shape)
    #print("OUTPUT GRADS:", test_grads.shape)
    #print(W_output_grad.shape)

    # Add one column of "1"s to X
    X_input = np.concatenate([np.ones((m, 1)), X], axis=1)

    ##########################################################################################
    # Full Mark: 3                                                                           #
    # TODO:                                                                                  #
    # 1. Transform y to one-hot encoding. Implement binary cross-entropy loss function       #
    # (Hint: Use the forward function to get necessary outputs from the model)               #
    #                                                                                        #
    # 2. Add L2 regularization to all weights in loss                                        #
    # (Note that we will not add regularization to bias)                                     #
    #                                                                                        #
    # 3. Compute the gradient for W_hidden and W_output (including both weights and biases)  #
    # (Hint: use chain rule, and the sigmoid gradient, tanh gradient you have                #
    # implemented above. Don't forget to add the gradient of regularization term)            #
    ##########################################################################################

    # one hot encoding
    classes = len(list(set(y))) # number of classes for one hot encoding
    #y_hot = np.eye(classes)[y]
    y_hot = np.zeros((len(y), classes))
    for i in range(len(y)):
      y_hot[i, y[i]] = 1


    # cross entropy loss
    fwd = forward((W_hidden, W_output), X_input)
    A1 = fwd['a_hidden']
    A2 = fwd['a_output']
    z_output = fwd['z_output']
    z_hidden = fwd['z_hidden']
    #print(A2)
    #print(fwd['a_output'][1])
    L = (-1 / m) * np.sum(np.multiply(np.log(A2), y_hot) + np.multiply((1 - y_hot), np.log(1 - A2)))
    #L = (1 / m) * np.sum((-np.dot(y_hot, np.log(A2).T) - np.dot(1 - y_hot, np.log(1 - A2).T)))
    #print("Loss:", L)


    # add regularization to all weights in loss except bias -- bias is W[0, :]
    L += (1 / (2 * m)) * np.sum(np.power(W_hidden[1:, :], 2)) + (1 / (2 * m)) * np.sum(np.power(W_output[1:, :], 2))


    # compute gradient - W_hidden (5, 10) | W_output (11, 3)

    # L = -y_hot * log(A2) - (1 - y_hot) * log(1 - A2)
    # dL/A2 = -y_hot/A2 - (1 - y_hot)/(1 - A2)
    dLA2 = -y_hot/A2 - (1 - y_hot)/(1 - A2)
    # A2 = sigmoid(z_output)
    # dA2_z_output = sigmoid_gradient(z_output)
    # dL_z_output = dL_A2 * sigmoid_gradient(z_output)
    dLZoutput = np.dot(dLA2.T, sigmoid_gradient(z_output))
    # z_output = a_hidden * W_output
    # dZ_output_A1 = w_output
    # dL_A1 = w_output * dL_Z_output
    dLA1 = (1 / m) * np.dot(W_output, dLZoutput.T)
    #print(dLA1.shape) # (11, 3)
    w_output_grad = dLA1
    
    # A1 = tanh(z_hidden)
    # dA1_Z_hidden = tanh_gradient(z_hidden)
    # dL_z_hidden = dL_A1 * tanh_gradient(z_hidden)
    #print("dLA2", dLA2.shape)
    #print("dLZoutput", dLZoutput.shape)
    #print("dLA1", dLA1.shape)
    #print("X_input", X_input.shape)
    dLzHidden = np.dot(dLA1.T, tanh_gradient(A1).T)
    #print(dLzHidden.shape)

    # Z_hidden = w_hidden * X
    # dz_hidden_w_hidden = X
    # dL_w_hidden = X * dL_z_hidden

    #dLwHidden = np.dot(X_input, dLzHidden)
    #w_hidden_grad = dLwHidden

    dZ2 = A2 - y_hot
    dW2 = (1 / m) * np.dot(A1.T, dZ2)
    #print(dW2.shape)
    #W_hidden_grad = tanh_gradient(W_hidden)
    W_output_grad = dW2
    #W_hidden_grad = np.dot(np.dot((y - fwd['a_output']).T, sigmoid_gradient(W_output).T), tanh_gradient(W_hidden).T)
    #print("A1", A1.shape)
    #print("dZ2", dZ2.shape)
    #print("W_hidden", W_hidden.shape)
    #print("W_output", W_output.shape)
    dZ1 = np.multiply(np.dot(W_output, dZ2.T).T, tanh_gradient(A1))
    dW1 = (1 / m) * np.dot(X_input.T, dZ1[:, 1:])
    #print(dW1.shape)
    #W_output_grad = sigmoid_gradient(W_output)
    W_hidden_grad = dW1
    #W_output_grad = np.dot((y - fwd['a_output']).T, sigmoid_gradient(W_output).T)

    #print("RESULTING W HIDDEN GRAD:", W_hidden_grad.shape)
    #print("RESULTING W OUTPUT GRAD:", W_output_grad.shape)

    
    ###################################################################################
    #                       END OF YOUR CODE                                          #
    ###################################################################################

    grads = np.concatenate([W_hidden_grad.ravel(), W_output_grad.ravel()])
    #print("RESULTING GRADS:", grads.shape)

    return L, grads

In [196]:
def optimize(initial_W, X, y, num_epoch, num_feature, num_hidden, num_output, L2_lambda):
    '''
    :param initial_W: initial weights as a 1D array.
    :param X: input
    :param y: labels of X
    :param num_epoch: number of iterations
    :param num_feature: number of features in X
    :param num_hidden: number of hidden units
    :param num_output: number of output units
    :param L2_lambda: the coefficient of regularization term
    '''

    options = {'maxiter': num_epoch}

    ###########################################################################################
    # Full Mark: 1                                                                            #
    # TODO:                                                                                   #
    # Optimize weights                                                                        #
    # (Hint: use scipy.optimize.minimize to automatically do the iteration.)                  #
    # ref: https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html) #
    # For some optimizers, you need to set 'jac' as True.                                     #
    # You may need to use lambda to create a function with one parameter to wrap the          #
    # loss_funtion you have implemented above.                                                #
    #                                                                                         #
    # Note that scipy.optimize.minimize only accepts a 1D weight array as initial weights,    #
    # and the output optimized weights will also be a 1D array.                               #
    # That is why we unroll the initial weights into a single long vector.                    #
    ###########################################################################################

    # W, X, y, num_feature, num_hidden, num_output, L2_lambda
    loss = lambda w: loss_function(w, X, y, num_feature, num_hidden, num_output, L2_lambda)
    #W_final = minimize(fun = loss_function, x0 = initial_W, args = (X, y, num_feature, num_hidden, num_output, L2_lambda), 
    #                   jac = True, options = options)
    optimized = minimize(fun = loss, x0 = initial_W, 
                       jac = True, options = options)
    
    #print(optimized)
    
    W_final = optimized.x

    ###################################################################################
    #                       END OF YOUR CODE                                          #
    ###################################################################################

    # Obtain W_hidden and W_output back from W_final
    #print(W_final)
    W_hidden = np.reshape(W_final[:num_hidden * (num_feature + 1)],
                          ((num_feature + 1), num_hidden))
    W_output = np.reshape(W_final[(num_hidden * (num_feature + 1)):],
                          ((num_hidden + 1), num_output))

    return [W_hidden, W_output]

In [197]:
def predict(X_test, y_test, W):
    '''
    :param X_test: input
    :param y_test: labels of X_test
    :param W: a list containing two weights W_hidden and W_output.
    '''

    test_input = np.concatenate([np.ones((y_test.size, 1)), X_test], axis=1)

    ###################################################################################
    # Full Mark: 1                                                                    #
    # TODO:                                                                           #
    # Predict on test set and compute the accuracy.                                   #
    # (Hint: use forward function to get predicted output)                            #
    #                                                                                 #
    ###################################################################################

    fwd = forward(W, test_input)
    pred_class = []
    for res in fwd['a_output']:
      pred = np.argmax(res, axis = 0)
      pred_class.append(pred)
    #print(y_test)
    #print(fwd['a_output'])
    #print(pred_class)
    correct = 0
    for i in range(len(pred_class)):
      if pred_class[i] == y_test[i]:
        correct += 1
    acc = correct / len(y_test)

    ###################################################################################
    #                       END OF YOUR CODE                                          #
    ###################################################################################

    return acc

In [199]:
# Do not modify this part #
# Define parameters
NUM_FEATURE = 4
NUM_HIDDEN_UNIT = 10
NUM_OUTPUT_UNIT = 3
NUM_EPOCH = 100
L2_lambda = 1

# Load data
X, y = load_dataset()
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Initialize weights
initial_W_hidden = init_weights(NUM_FEATURE, NUM_HIDDEN_UNIT)
initial_W_output = init_weights(NUM_HIDDEN_UNIT, NUM_OUTPUT_UNIT)
initial_W = np.concatenate([initial_W_hidden.ravel(), initial_W_output.ravel()], axis=0)

# Neural network learning
W = optimize(initial_W, X_train, y_train, NUM_EPOCH, NUM_FEATURE, NUM_HIDDEN_UNIT, NUM_OUTPUT_UNIT, L2_lambda)

# Predict
acc = predict(X_test, y_test, W)
print("Test accuracy:", acc)

Test accuracy: 0.96
